In [1]:
%cd C:\Users\Admin\chatbot_on_apparels

C:\Users\Admin\chatbot_on_apparels


# NEW CODE

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
import pandas as pd
import calendar
import datetime as dt
from datetime import datetime
from datetime import date
from datetime import timedelta
import re
from fuzzywuzzy import fuzz, process


C:\Users\Admin\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
date_parser = lambda x: pd.to_datetime(x).date()

orders_df = pd.read_csv('orders_data.csv',parse_dates = ['expected_date_of_delivery',
                                                         'expected_date_of_refund',
                                                         'expected_date_of_return_pickup',
                                                          'expected_date_of_replacement',
                                                          'updated_status_date'],
                                                          date_parser = date_parser)
orders_df['order_id'] = orders_df['order_id'].astype(str)
#orders_df


C:\Users\Admin\AppData\Local\Temp\ipykernel_8592\2439922231.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  orders_df = pd.read_csv('orders_data.csv',parse_dates = ['expected_date_of_delivery',
C:\Users\Admin\AppData\Local\Temp\ipykernel_8592\2439922231.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date_parser = lambda x: pd.to_datetime(x).date()


In [4]:
products_df = pd.read_csv('products_data.csv')
products_df

,product_id,product_name,category,category_broad,details,price,offer,status
0,P1000,Casual Shoes 1,Casual Shoes,footwear,Canvas sneakers perfect for casual outings,2293,Flat 50% Off,offer/sale/discount
1,P1001,T-Shirts 2,Tshirts,clothing,Durable polyester blend for active lifestyles,3485,No discount,fresh_arrivals
2,P1002,Jackets 3,Jackets,clothing,Genuine leather jacket with a rugged look,3420,No discount,Limited Edition
3,P1003,Boxers 4,Boxers,clothing,Blended material boxers with a snug fit,2404,No discount,fresh_arrivals
4,P1004,T-Shirts 5,Tshirts,clothing,Casual and comfortable blended material,1572,No discount,fresh_arrivals
...,...,...,...,...,...,...,...,...
995,P1995,Sweaters 996,Sweaters,clothing,Warm wool knit for chilly days,1506,No discount,fresh_arrivals
996,P1996,Sandals 997,Sandals,footwear,Comfortable synthetic sandals for casual wear,2048,No discount,fresh_arrivals
997,P1997,Jackets 998,Jackets,clothing,Water-resistant synthetic material for all wea...,4310,Clearance Sale,offer/sale/discount
998,P1998,Jeans 999,Jeans,clothing,Stretch denim for added comfort and flexibility,4108,20% Off,offer/sale/discount


In [5]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Load the intents file
with open('intents_consol.json') as file:
    intents = json.load(file)

lemmatizer = WordNetLemmatizer()

# Preprocess data
corpus = []
labels = []
classes = []
responses = {}




for intent in intents['intents']:
    for pattern in intent['patterns']:
        tokens = nltk.word_tokenize(pattern)
        corpus.append(' '.join([lemmatizer.lemmatize(token.lower()) for token in tokens]))
        labels.append(intent['tag'])
    
    responses[intent['tag']] = intent['responses']
    classes.append(intent['tag'])

vectorizer = TfidfVectorizer(max_features=2000)
X = vectorizer.fit_transform(corpus)
y = np.array(labels)

# Train the model
model = LinearSVC()
model.fit(X, y)

def validate_order_id(orderid):
    orderid = str(orderid)
    return orderid in orders_df['order_id'].values

def get_order_details(orderid):
    return orders_df.loc[orders_df['order_id'] == orderid].iloc[0]

def is_valid_date(date_string): 
    try: 
        datetime.strptime(date_string, '%Y-%m-%d') 
        return True 
    except ValueError: 
        return False
    
##################################################################################


# List of keywords
keywords = [
    "Casual Shoes", "Tshirts", "Jackets", "Boxers", "Sunglasses", "Shirts", 
    "Belts", "Shorts", "Watches", "Jeans", "Ethnic Wear", "Hoodies", 
    "Sandals", "Formal Shoes", "Sweaters"
]

# Join the keywords into a single regex pattern
keywords_pattern = re.compile(r'|'.join([re.escape(keyword) for keyword in keywords]), re.IGNORECASE)


# Regular expressions for different patterns
greater_than_pattern = re.compile(r'(?:>\s?\d+|gt\s?\d+|grt\s?\d+|grt\s?than\s?\d+|>\s?than\s?\d+|GREATER\s?THAN\s?\d+)', re.IGNORECASE)
less_than_pattern = re.compile(r'(?:<\s?\d+|lt\s?\d+|LT\s?\d+|lt\s?than\s?\d+|<\s?than\s?\d+|lESS\s?THAN\s?\d+)', re.IGNORECASE)
range_pattern = re.compile(r'(?:>\s?\d+\s?and\s?<\s?\d+|grt\s?\d+\s?lt\s?\d+|gt\s?\d+\s?lt\s?\d+|greater\s?than\s?\d+\s?AND\s?less\s?than\s?\d+|>\s?than\s?\d+\s?AND\s?<\s?than\s?\d+|between\s?\d+\s?and\s?\d+|BTW\s?\d+\s?AND\s?\d+|between\s?\d+\s?&\s?\d+|btw\s?\d+\s?&\s?\d+)', re.IGNORECASE)

# Function to extract values and keywords
def extract_min_max_category(text_input):
    matches = re.findall(r'\d+', text_input)
    keywords_found = re.findall(keywords_pattern, text_input)
    
    grt_than = less_than = None
    if re.search(range_pattern, text_input):
        if len(matches) == 2:
            grt_than, less_than = int(matches[0]), int(matches[1])
    elif re.search(greater_than_pattern, text_input):
        if len(matches) == 1:
            grt_than = int(matches[0])
    elif re.search(less_than_pattern, text_input):
        if len(matches) == 1:
            less_than = int(matches[0])
    
    # Fuzzy matching for keywords
    keywords_found = []
    for keyword in keywords:
        if process.extractOne(keyword, text_input.split(), scorer=fuzz.ratio)[1] >= 90:
            keywords_found.append(keyword)
    product_category = keywords_found[0] if keywords_found else 'all'
    
    
    grt_than = 0 if grt_than is None else grt_than
    less_than = 0 if less_than is None else less_than
    return grt_than, less_than, product_category


def product_price_range_details(max_val,min_val,category_value):
    if max_val > 0 and min_val == 0 and category_value == 'all':
        product_price_range_df = products_df.loc[products_df['price'] > max_val, ['product_id', 'product_name', 'price']]
        records_count = product_price_range_df.shape[0]
    elif max_val == 0 and min_val > 0 and category_value == 'all':
        product_price_range_df = products_df.loc[products_df['price'] < min_val, ['product_id', 'product_name', 'price']]
        records_count = product_price_range_df.shape[0]
    elif max_val > 0 and min_val > 0 and category_value == 'all':
        product_price_range_df = products_df.loc[(products_df['price'] > max_val) & (products_df['price'] < min_val), ['product_id', 'product_name', 'price']]
        records_count = product_price_range_df.shape[0]
    elif max_val > 0 and min_val == 0 and category_value != 'all':
        product_price_range_df = products_df.loc[(products_df['price'] > max_val) & (products_df['category'].str.lower() == category_value.lower()), ['product_id', 'product_name', 'price']]
        records_count = product_price_range_df.shape[0]
    elif max_val == 0 and min_val > 0 and category_value != 'all':
        product_price_range_df = products_df.loc[(products_df['price'] < min_val) & (products_df['category'].str.lower() == category_value.lower()), ['product_id', 'product_name', 'price']]
        records_count = product_price_range_df.shape[0]
    elif max_val > 0 and min_val > 0 and category_value != 'all':
        product_price_range_df = products_df.loc[(products_df['price'] > max_val) & (products_df['price'] < min_val) & (products_df['category'].str.lower() == category_value.lower()), ['product_id', 'product_name', 'price']]
        records_count = product_price_range_df.shape[0]
    else:
        print (f"not able to get the desired data as per the inputs")
    return product_price_range_df,category_value,records_count
        
def product_details():
    distinct_products = products_df['category'].unique()
    comma_separated_distinct_products = ', '.join(distinct_products)
    return comma_separated_distinct_products

def product_wise_details(category):
    product_category_df1 = products_df[(products_df['category'].str.lower() == category)]
    product_category_df = product_category_df1[['product_id', 'product_name', 'details']]
    product_category_df = product_category_df.head()
    return product_category_df


def price_details():
    min_price = products_df['price'].min()
    max_price = products_df['price'].max()
    return min_price,max_price

def product_price_wise_details(category):
    product_category_price_df1 = products_df[(products_df['category'].str.lower() == category)]
    product_category_price_df = product_category_price_df1[['product_id', 'product_name', 'price']]
    product_category_price = product_category_price_df.head()
    min_category_price = product_category_price_df['price'].min()
    max_category_price = product_category_price_df['price'].max()
    return product_category_price,min_category_price,max_category_price


def offer_details():
    distinct_offers = products_df[(products_df['offer']!= 'No discount')]['offer'].unique()
    comma_separated_distinct_offers = ', '.join(distinct_offers)
    return comma_separated_distinct_offers


def product_offer_wise_details(category):
    product_category_offer_price_df1 = products_df[(products_df['category'].str.lower() == category)]
    product_category_offer_price_df = product_category_offer_price_df1[['product_id', 'product_name', 'price','offer']]
    product_category_offer_price = product_category_offer_price_df.head()
    distinct_category_offers = product_category_offer_price_df[(product_category_offer_price_df['offer']!= 'No discount')]['offer'].unique()
    comma_separated_distinct_category_offers = ', '.join(distinct_category_offers)
    return product_category_offer_price,comma_separated_distinct_category_offers




def chatbot_response(text):
    tokens = nltk.word_tokenize(text)
    input_vec = vectorizer.transform([' '.join([lemmatizer.lemmatize(token.lower()) for token in tokens])])
    prediction = model.predict(input_vec)[0]
    
    
    if prediction == 'product_details':
        product_category_master = product_details()
        return f"We sell products in the following categories :'{product_category_master}'."
    
    if prediction == 'price_details':
        min_products_price,max_products_price = price_details()
        return f"We sell products in men's clothing,footwear & accessories categories in the price range of Rs {min_products_price}' to Rs {max_products_price}."
        
     
    if prediction == 'offer' or prediction == 'discount':
        offer_master = offer_details()
        return f"The current offers/discounts available are : {offer_master}."
    
    if prediction in ['casual_shoes_details','tshirts_details','jackets_details','boxers_details','sunglasses_details',
                      'shirts_details','belts_details','shorts_details','watches_details','jeans_details',
                      'ethnic_wear_details','hoodies_details','sandals_details','formal_shoes_details','sweaters_details'
    ]:
        product_category_split = prediction.split('_')
        product_category_actual = ' '.join(product_category_split[:-1])
        product_category_specific_df = product_wise_details(product_category_actual)
        return (f"below are the details of Top5 products in {product_category_actual} category :\n" 
                f"{product_category_specific_df}")
               
 
    if prediction in ['casual_shoes_price','tshirts_price','jackets_price','boxers_price','sunglasses_price',
                      'shirts_price','belts_price','shorts_price','watches_price','jeans_price',
                      'ethnic_wear_price','hoodies_price','sandals_price','formal_shoes_price','sweaters_price'
    ]:
        product_category_price_split = prediction.split('_')
        product_category_price_actual = ' '.join(product_category_price_split[:-1])
        product_category_price_specific_df,min_category_price,max_category_price = product_price_wise_details(product_category_price_actual)
        return (f"below are the details of the Top 5 products in the {product_category_price_actual} category:\n" 
                f"{product_category_price_specific_df}\n\n{product_category_price_actual}" 
                f" in the price range of Rs {min_category_price} to Rs {max_category_price} are available.")

    
    if prediction in ['casual_shoes_offer','tshirts_offer','jackets_offer','boxers_offer','sunglasses_offer',
                      'shirts_offer','belts_offer','shorts_offer','watches_offer','jeans_offer',
                      'ethnic_wear_offer','hoodies_offer','sandals_offer','formal_shoes_offer','sweaters_offer'
    ]:
        product_category_offer_split = prediction.split('_')
        product_category_offer_actual = ' '.join(product_category_offer_split[:-1])
        product_category_offer_specific_df,category_specific_offers = product_offer_wise_details(product_category_offer_actual)
        return (f"below are the details of the Top 5 products in the {product_category_offer_actual} category:\n"
                f"{product_category_offer_specific_df}\n\nThe following offers are there in {product_category_offer_actual}" 
                f"category :\n {category_specific_offers}") 

    if prediction in ['price_range']:
        max_value,min_value,category = extract_min_max_category(text)
        product_price_range_df_data,cat_value,row_count = product_price_range_details(max_value,min_value,category)
        return (f"below are the details of the products in the {cat_value} category:\n"
                f"{product_price_range_df_data}\n\n"
                f"There are {row_count} {cat_value} found in your chosen price range")
    
    
    elif prediction in ['order_status', 'expected_delivery_time', 'expected_date_of_delivery',
                        'expected_date_of_refund', 'expected_date_of_return', 'expected_date_of_replacement',
                        'order_cancellation', 'order_rescheduling', 'other_request','complaint_existing'
    ]:
        
        print (f'######################################################################################################')
        orderid = input("Please enter your order ID or type 'exit' to go to the main menu: ")
        if orderid.lower() == "exit":
            return "Returning to the main menu."
        while not validate_order_id(orderid):
            print (f'######################################################################################################')
            orderid = input("Invalid order ID. Please enter a valid order ID or type 'exit' to go to the main menu: ")
            if orderid.lower() == "exit":
                return "Returning to the main menu."
        order_details = get_order_details(orderid)
        
        
        # Handle different cases after order ID validation
        if prediction == 'order_status':
            return f"Your order status is '{order_details['status']}'."
        
        elif prediction == 'expected_delivery_time':
            return f"Your expected delivery date is '{order_details['expected_date_of_delivery']}'."
        
        elif prediction == 'expected_date_of_delivery':
            return f"Your expected delivery date is '{order_details['expected_date_of_delivery']}'."
        
        elif prediction == 'expected_date_of_refund':
            return f"Your expected date of refund is '{order_details['expected_date_of_refund']}'."
        
        elif prediction == 'expected_date_of_return':
            return f"Your expected date of return pickup is '{order_details['expected_date_of_return_pickup']}'."
        
        elif prediction == 'expected_date_of_replacement':
            return f"Your expected date of replacement delivery is '{order_details['expected_date_of_replacement']}'."
        
        elif prediction == 'order_cancellation':
            if order_details['status'] in ['Order Placement','Order Confirmation'] and order_details['updated_status'] not in ['order cancelled','order rescheduled']:
                print (f'######################################################################################################')
                update_reason = input("Please provide reason for cancellation : ")
                orders_df.loc[orders_df['order_id'] == orderid, 'updated_status'] =  'order cancelled'
                orders_df.loc[orders_df['order_id'] == orderid, 'updated_status_date'] =  pd.to_datetime(datetime.today())
                orders_df.loc[orders_df['order_id'] == orderid, 'updated_status_reason'] =  update_reason
                order_details['updated_status'] =  'order cancelled'
                order_details['updated_status_date'] =  pd.to_datetime(datetime.today())
                order_details['updated_status_reason'] =  update_reason
                
                return "Your order has been cancelled and the refund will be processed within 5-7 business days."
                
            if order_details['updated_status'] == "" or order_details['updated_status'] == "na":
                return f"Your order can't be cancelled as it is already in '{order_details['status']}' stage."
            return f"Your order can't be cancelled as it is already in '{order_details['updated_status']}' stage."
    
        
    
        elif prediction == 'order_rescheduling':
            if order_details['status'] in ['Order Placement','Order Confirmation'] and order_details['updated_status'] not in ['order cancelled','order rescheduled']:

                while True:
                    try:
                        while True:
                            print (f'######################################################################################################')
                            new_date = input("Please provide the new date for rescheduling (YYYY-MM-DD) or type 'exit' to go to the main menu: ")
                            if new_date.lower() == "exit":
                                return "Returning to the main menu."
                            if not is_valid_date(new_date):
                                print("Invalid date format. Please enter the date in YYYY-MM-DD format.")
                                continue
                            reschedule_date = pd.to_datetime(new_date)
                            day_diff = (reschedule_date - order_details['expected_date_of_delivery']).days
                            order_details['day_diff'] = day_diff
                            if order_details['day_diff'] <=3:
                                print (f'######################################################################################################')
                                update_reason = input("Please provide reason for rescheduling the order : ")
                                orders_df.loc[orders_df['order_id'] == orderid, 'updated_status'] =  'order rescheduled'
                                orders_df.loc[orders_df['order_id'] == orderid, 'updated_status_date'] =  reschedule_date
                                orders_df.loc[orders_df['order_id'] == orderid, 'updated_status_reason'] =  update_reason
                                order_details['updated_status'] =  'order rescheduled'
                                order_details['updated_status_date'] =  reschedule_date
                                order_details['updated_status_reason'] =  update_reason
                                return f"Your order has been rescheduled to {reschedule_date}."
                            else:
                                print (f'######################################################################################################')
                                print("The reschedule date can't be more than 3 days from the expected delivery date")
                    except ValueError:
                        print (f'######################################################################################################')
                        print("Invalid date format. Please enter the date in YYYY-MM-DD format.")
                    
                     
            
            if order_details['updated_status'] == "" or order_details['updated_status'] == "na":
                return f"Your order can't be rescheduled as it is already in '{order_details['status']}' stage."
            return f"Your order can't be rescheduled as it is already in '{order_details['updated_status']}' stage."
        
            
            
        elif prediction == 'other_request':
            return "Please provide details of your request so we can assist you further."
        elif prediction == 'complaint_existing':
            print (f'######################################################################################################')
            complaint = input("Please describe the issue in detail or type 'exit' to go to the main menu: ")
            if complaint.lower() == "exit":
                return "Returning to the main menu."
            return f"{random.choice(responses[prediction])}\nThank you for your input.\n Our customer care executive will call you back within 2 hours."
            
    else:
        return "I'm sorry, I didn't understand that. Can you please rephrase?"

# Chat with the bot
print (f'######################################################################################################')
print("Hello! How can I assist you today?")
while True:
    print('\n')
    print (f'######################################################################################################')
    user_input = input("You: ")
    print (f'######################################################################################################')
    if user_input.lower() == "exit" or user_input.lower() == "exit":
        break
    response = chatbot_response(user_input)
    orders_df['order_id'] = orders_df['order_id'].astype(str)  # Convert 'A' to integer to avoid type mismatch
    print(f"Bot: {response}")
    if 'Goodbye' in response:
        break
    print('\n')
    print("Hello! How can I assist you today?")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


######################################################################################################
Hello! How can I assist you today?


######################################################################################################
You: belts less than 1000
######################################################################################################
Bot: below are the details of the products in the Belts category:
    product_id product_name  price
94       P1094     Belts 95    800
206      P1206    Belts 207    906
383      P1383    Belts 384    639
406      P1406    Belts 407    921
579      P1579    Belts 580    767
595      P1595    Belts 596    611
717      P1717    Belts 718    708
752      P1752    Belts 753    850
808      P1808    Belts 809    609

There are 9 Belts found in your chosen price range


Hello! How can I assist you today?


######################################################################################################
You: belts greater

In [6]:
orders_df[(orders_df['order_id'] == '5919')]

,order_id,status,expected_date_of_delivery,expected_date_of_refund,expected_date_of_return_pickup,expected_date_of_replacement,updated_status,updated_status_date,updated_status_reason
24,5919,Order Placement,2025-01-10,NaT,NaT,NaT,order cancelled,2025-01-09,NaN


In [1]:
import re

# List of keywords
keywords = [
    "CasualShoes", "Tshirts", "Jackets", "Boxers", "Sunglasses", "Shirts", 
    "Belts", "Shorts", "Watches", "Jeans", "Ethnic Wear", "Hoodies", 
    "Sandals", "Formal Shoes", "Sweaters"
]

# Join the keywords into a single regex pattern
keywords_pattern = re.compile(r'|'.join([re.escape(keyword) for keyword in keywords]), re.IGNORECASE)

# Sample text inputs
inputs = [
    "price >50 and <100 with CasualShoes and Tshirts",
    "price grt 50 lt 100 with Jackets and Boxers",
    "price greater than 50 less than 100 with Sunglasses and Shirts",
    "between 50 and 100 with Belts and Shorts",
    "between 50 100 with Watches and Jeans",
    "price >50 with Hoodies",
    "price <50 with Sandals and Formal Shoes",
    "price >50",  # No keywords
    "price <50"   # No keywords
]

# Regular expressions for different patterns
greater_than_pattern = re.compile(r'(?:>\s?\d+|gt\s?\d+|grt\s?\d+|grt\s?than\s?\d+|>\s?than\s?\d+|GREATER\s?THAN\s?\d+)', re.IGNORECASE)
less_than_pattern = re.compile(r'(?:<\s?\d+|lt\s?\d+|LT\s?\d+|lt\s?than\s?\d+|<\s?than\s?\d+|lESS\s?THAN\s?\d+)', re.IGNORECASE)
range_pattern = re.compile(r'(?:>\s?\d+\s?and\s?<\s?\d+|grt\s?\d+\s?lt\s?\d+|gt\s?\d+\s?lt\s?\d+|greater\s?than\s?\d+\s?AND\s?less\s?than\s?\d+|>\s?than\s?\d+\s?AND\s?<\s?than\s?\d+|between\s?\d+\s?and\s?\d+|BTW\s?\d+\s?AND\s?\d+|between\s?\d+\s?&\s?\d+|btw\s?\d+\s?&\s?\d+)', re.IGNORECASE)

# Function to extract values and keywords
def extract_values_and_keywords(text):
    matches = re.findall(r'\d+', text)
    keywords_found = re.findall(keywords_pattern, text)
    
    A = B = None
    if re.search(range_pattern, text):
        if len(matches) == 2:
            A, B = int(matches[0]), int(matches[1])
    elif re.search(greater_than_pattern, text):
        if len(matches) == 1:
            A = int(matches[0])
    elif re.search(less_than_pattern, text):
        if len(matches) == 1:
            B = int(matches[0])
    
    C = keywords_found[0] if keywords_found else 'ALL'
    return A, B, C

# Extract values and keywords from inputs
for text in inputs:
    A, B, C = extract_values_and_keywords(text)
    print(f"Input: '{text}'; A: {A}, B: {B}; Keyword: {C}")


Input: 'price >50 and <100 with CasualShoes and Tshirts'; A: 50, B: 100; Keyword: CasualShoes
Input: 'price grt 50 lt 100 with Jackets and Boxers'; A: 50, B: 100; Keyword: Jackets
Input: 'price greater than 50 less than 100 with Sunglasses and Shirts'; A: None, B: None; Keyword: Sunglasses
Input: 'between 50 and 100 with Belts and Shorts'; A: 50, B: 100; Keyword: Belts
Input: 'between 50 100 with Watches and Jeans'; A: None, B: None; Keyword: Watches
Input: 'price >50 with Hoodies'; A: 50, B: None; Keyword: Hoodies
Input: 'price <50 with Sandals and Formal Shoes'; A: None, B: 50; Keyword: Sandals
Input: 'price >50'; A: 50, B: None; Keyword: ALL
Input: 'price <50'; A: None, B: 50; Keyword: ALL


In [14]:
orders_df.to_csv('orders_data.csv',index = False)

In [6]:
import re
from fuzzywuzzy import fuzz, process

# List of keywords
keywords = [
    "CasualShoes", "Tshirts", "Jackets", "Boxers", "Sunglasses", "Shirts", 
    "Belts", "Shorts", "Watches", "Jeans", "Ethnic Wear", "Hoodies", 
    "Sandals", "Formal Shoes", "Sweaters"
]

# Sample text inputs
inputs = [
    "price >50 and <100 with CasualShoes and Tshirts",
    "price grt 50 lt 100 with Jakets and Boxrs",  # intentionally misspelled
    "price greater than 50 less than 100 with Sunglasses and Shirts",
    "between 50 and 100 with Belts and Shorts",
    "between 50 100 with Watches and Jeans",
    "price >50 with Hoodies",
    "price <50 with Sandals and Formal Shoes",
    "price >50",  # No keywords
    "price <50"   # No keywords
]

# Regular expressions for different patterns
greater_than_pattern = re.compile(r'(?:>\s?\d+|gt\s?\d+|grt\s?\d+|grt\s?than\s?\d+|>\s?than\s?\d+|GREATER\s?THAN\s?\d+)', re.IGNORECASE)
less_than_pattern = re.compile(r'(?:<\s?\d+|lt\s?\d+|LT\s?\d+|lt\s?than\s?\d+|<\s?than\s?\d+|lESS\s?THAN\s?\d+)', re.IGNORECASE)
range_pattern = re.compile(r'(?:>\s?\d+\s?and\s?<\s?\d+|grt\s?\d+\s?lt\s?\d+|gt\s?\d+\s?lt\s?\d+|greater\s?than\s?\d+\s?AND\s?less\s?than\s?\d+|>\s?than\s?\d+\s?AND\s?<\s?than\s?\d+|between\s?\d+\s?and\s?\d+|BTW\s?\d+\s?AND\s?\d+|between\s?\d+\s?&\s?\d+|btw\s?\d+\s?&\s?\d+)', re.IGNORECASE)

# Function to extract values and keywords
def extract_values_and_keywords(text):
    matches = re.findall(r'\d+', text)
    
    A = B = None
    if re.search(range_pattern, text):
        if len(matches) == 2:
            A, B = int(matches[0]), int(matches[1])
    elif re.search(greater_than_pattern, text):
        if len(matches) == 1:
            A = int(matches[0])
    elif re.search(less_than_pattern, text):
        if len(matches) == 1:
            B = int(matches[0])
    
    # Fuzzy matching for keywords
    keywords_found = []
    for keyword in keywords:
        if process.extractOne(keyword, text.split(), scorer=fuzz.ratio)[1] >= 80:
            keywords_found.append(keyword)
    
    C = keywords_found[0] if keywords_found else 'ALL'
    return A, B, C

# Extract values and keywords from inputs
for text in inputs:
    A, B, C = extract_values_and_keywords(text)
    A = 0 if A is None else A
    B = 0 if B is None else B
    print(f"Input: '{text}'; A: {A}, B: {B}; Keyword: {C}")


Input: 'price >50 and <100 with CasualShoes and Tshirts'; A: 50, B: 100; Keyword: CasualShoes
Input: 'price grt 50 lt 100 with Jakets and Boxrs'; A: 50, B: 100; Keyword: Jackets
Input: 'price greater than 50 less than 100 with Sunglasses and Shirts'; A: 0, B: 0; Keyword: Tshirts
Input: 'between 50 and 100 with Belts and Shorts'; A: 50, B: 100; Keyword: Shirts
Input: 'between 50 100 with Watches and Jeans'; A: 0, B: 0; Keyword: Watches
Input: 'price >50 with Hoodies'; A: 50, B: 0; Keyword: Hoodies
Input: 'price <50 with Sandals and Formal Shoes'; A: 0, B: 50; Keyword: Sandals
Input: 'price >50'; A: 50, B: 0; Keyword: ALL
Input: 'price <50'; A: 0, B: 50; Keyword: ALL


In [2]:
import re
from fuzzywuzzy import fuzz, process

# List of keywords
keywords = [
    "CasualShoes", "Tshirts", "Jackets", "Boxers", "Sunglasses", "Shirts", 
    "Belts", "Shorts", "Watches", "Jeans", "Ethnic Wear", "Hoodies", 
    "Sandals", "Formal Shoes", "Sweaters"
]

# Sample text inputs
inputs = [
    "price >50 and <100 with CasualShoes and Tshirts",
    "price grt 50 lt 100 with Jakets and Boxrs",  # intentionally misspelled
    "price greater than 50 less than 100 with Sunglasses and Shirts",
    "between 50 and 100 with Belts and Shorts",
    "between 50 100 with Watches and Jeans",
    "price >50 with Hoodies",
    "price <50 with Sandals and Formal Shoes",
    "price >50",  # No keywords
    "price <50"   # No keywords
]

# Regular expressions for different patterns
greater_than_pattern = re.compile(r'(?:>\s?\d+|gt\s?\d+|grt\s?\d+|grt\s?than\s?\d+|>\s?than\s?\d+|GREATER\s?THAN\s?\d+)', re.IGNORECASE)
less_than_pattern = re.compile(r'(?:<\s?\d+|lt\s?\d+|LT\s?\d+|lt\s?than\s?\d+|<\s?than\s?\d+|lESS\s?THAN\s?\d+)', re.IGNORECASE)
range_pattern = re.compile(r'(?:>\s?\d+\s?and\s?<\s?\d+|grt\s?\d+\s?lt\s?\d+|gt\s?\d+\s?lt\s?\d+|greater\s?than\s?\d+\s?AND\s?less\s?than\s?\d+|>\s?than\s?\d+\s?AND\s?<\s?than\s?\d+|between\s?\d+\s?and\s?\d+|BTW\s?\d+\s?AND\s?\d+|between\s?\d+\s?&\s?\d+|btw\s?\d+\s?&\s?\d+)', re.IGNORECASE)

# Function to extract values and keywords
def extract_values_and_keywords(text):
    matches = re.findall(r'\d+', text)
    
    A = B = None
    if re.search(range_pattern, text):
        if len(matches) == 2:
            A, B = int(matches[0]), int(matches[1])
    elif re.search(greater_than_pattern, text):
        if len(matches) == 1:
            A = int(matches[0])
    elif re.search(less_than_pattern, text):
        if len(matches) == 1:
            B = int(matches[0])
    
    # Fuzzy matching for keywords
    keywords_found = []
    for keyword in keywords:
        if process.extractOne(keyword, text.split(), scorer=fuzz.ratio)[1] >= 90:
            keywords_found.append(keyword)
    
    C = f"'{keywords_found[0]}'" if keywords_found else "'ALL'"
    return A, B, C

# Extract values and keywords from inputs
for text in inputs:
    A, B, C = extract_values_and_keywords(text)
    A = 0 if A is None else A
    B = 0 if B is None else B
    print(f"Input: '{text}'; A: {A}, B: {B}; Keyword: {C}")


Input: 'price >50 and <100 with CasualShoes and Tshirts'; A: 50, B: 100; Keyword: 'CasualShoes'
Input: 'price grt 50 lt 100 with Jakets and Boxrs'; A: 50, B: 100; Keyword: 'Jackets'
Input: 'price greater than 50 less than 100 with Sunglasses and Shirts'; A: 0, B: 0; Keyword: 'Tshirts'
Input: 'between 50 and 100 with Belts and Shorts'; A: 50, B: 100; Keyword: 'Shirts'
Input: 'between 50 100 with Watches and Jeans'; A: 0, B: 0; Keyword: 'Watches'
Input: 'price >50 with Hoodies'; A: 50, B: 0; Keyword: 'Hoodies'
Input: 'price <50 with Sandals and Formal Shoes'; A: 0, B: 50; Keyword: 'Sandals'
Input: 'price >50'; A: 50, B: 0; Keyword: 'ALL'
Input: 'price <50'; A: 0, B: 50; Keyword: 'ALL'
